In [ ]:
# @title
import os,subprocess,sys,fileinput,time,requests,json,subprocess,gc,threading,time,re,socket,urllib.request,ipywidgets as widgets
from datetime import datetime
from subprocess import getoutput
from IPython.utils import capture
from PIL import Image
from IPython.display import clear_output,display, HTML, Image
from io import BytesIO
import zipfile, random, string
!pip install python-multipart
varsayilan_degerler = {
    'SurucuSenkron' : True,
    'Ayar_Onceden_Secili' : "Yok",
    'Kutuhane' : True,
    'HizliMod': True,
    'ana_klasor' :'/content',
    'Kullanici_klasoru' : '/content/drive/MyDrive/TSD-Data',
    'API_klasoru': '/content/TSD-WebUI',
    'TSDKlasoru': '/content/TSD-WebUI',
    'Versiyon':'ComfyUI',
    'SecenemModu':'temel',
    'KontrolAgi':'yok',
    'SDmodeli':'',
    'SD15modeli':'',
    'SDXLmodeli':'',
    'Fluxmodeli':'',
    'TSDmodeli':'',
    'InpaintModeli':'',
    'SurucuKutuphanesi' : True,
    'KomutSatiri' : '',
    'arayuz_klasoru': '/usr/local/lib/python3.12/dist-packages/comfyui_frontend_package/static',
}
if 'ArayuzVersiyon' in globals():
  Versiyon = ArayuzVersiyon
for anahtar, deger in varsayilan_degerler.items():
  if anahtar not in globals():
    globals()[anahtar] = deger
  elif type(globals()[anahtar]) == str:
    globals()[anahtar] = globals()[anahtar].split(' ')[-1]

def dosya_metni_degistir(dosya_yolu, eski_metin, yeni_metin):
      with open(dosya_yolu, 'r', encoding='utf-8') as dosya:
          dosya_icerigi = dosya.read()
      guncellenen_icerik = dosya_icerigi.replace(eski_metin, yeni_metin)
      with open(dosya_yolu, 'w', encoding='utf-8') as dosya:
          dosya.write(guncellenen_icerik)

def bilgi(mesaj, stil, genislik):
    bilgi_buton = widgets.Button(description=mesaj, disabled=True, button_style=stil, layout=widgets.Layout(min_width=genislik))
    display(bilgi_buton)

def aria_indir(baglanti, yol, isim):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {baglanti} -d  {yol} -o {isim}

def klasor_olustur(yol):
  yol = f'{Kullanici_klasoru}/{yol}'
  os.makedirs(yol, exist_ok=True)

def varsayilan_dosya_kontrol(kaynak_klasor, kontrol_klasor, uzerine_yaz=False):
  for oge in os.listdir(kaynak_klasor):
    dosya_yolu = os.path.join(kaynak_klasor, oge)
    if os.path.isfile(dosya_yolu):
      kopyala_yolu = dosya_yolu.replace(TSDKlasoru, kontrol_klasor)
      !cp {'-f' if uzerine_yaz else '-n'} {dosya_yolu} {kopyala_yolu}   
    elif os.path.isdir(dosya_yolu):
      varsayilan_dosya_kontrol(dosya_yolu, kontrol_klasor, uzerine_yaz)

def civit_indirme_baglantisi(baglanti):
  !wget {baglanti} -q -O model.html
  try:
      with open('model.html', 'r', encoding='utf-8') as dosya:
          html_icerigi = dosya.read()
      desen = r'"modelVersionId":(\\d+),'
      model_id = re.findall(desen, html_icerigi)
      if model_id:
        api_baglantisi = f'https://civitai.com/api/download/models/{model_id[0]}'
        print(f'Model id indirme bağlantısı: {api_baglantisi}')
        return api_baglantisi
      else:
          return "Uygun içerik bulunamadı."
  except requests.RequestException as e:
      return f"Sayfa yükleme hatası: {e}"

def baglanti_kontrol(baglanti):
  if 'huggingface.co' in baglanti:
    if 'blob' in baglanti:
      baglanti = baglanti.replace('blob', 'resolve')
      return baglanti
    else:
      return baglanti
  if 'civitai.com' in baglanti:
    if 'civitai.com/models' in baglanti:
      return civit_indirme_baglantisi(baglanti)
    else:
      return baglanti
    
def token_al(baglanti):
  if "civitai" in baglanti:
    token = f'?token=8c7337ac0c39fe4133ae19a3d65b806f'
  else:
    token = ""
  return token

def liste_txt_calistir(txt):
  with open(f"{txt}", "r") as dosya:
    listeler = dosya.readlines()
  for liste in listeler :
    !{liste}

def klasor_bagla(kaynak, hedef):
  !rm -rf {hedef}
  !ln -s {kaynak} {hedef}

def kontrol_et(yol):
    if not os.path.exists(yol):
        os.makedirs(yol)

def baglanti_ayir(baglanti):
    ayirici = "@="
    if ayirici in baglanti:
        parcalar = baglanti.split(ayirici)
        if '.ckpt' not in parcalar[1] and '.gguf' not in parcalar[1] and '.safetensors' not in parcalar[1]:
          parcalar[1] = f'{parcalar[1]}.safetensors'
        return parcalar[0], parcalar[1]
    else:
        return baglanti, None
        
def indir(baglanti):
  indirme_kodu = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
  baglanti = baglanti.replace('&', '\\&')
  deger_baglanti, deger_isim = baglanti_ayir(baglanti)
  deger_baglanti = deger_baglanti.split("?")[0]
  deger_baglanti = baglanti_kontrol(deger_baglanti)
  if '#' not in baglanti:
    if deger_isim != None :
      if 'civit' not in baglanti:
        !{indirme_kodu} {deger_baglanti} -d {kontrol_nokta_yolu} -o {deger_isim}
      else:
        !wget {deger_baglanti}{token_al(deger_baglanti)} -O {kontrol_nokta_yolu}/{deger_isim}
    else:
      !wget {deger_baglanti}{token_al(deger_baglanti)} -P {kontrol_nokta_yolu} --content-disposition